In [70]:
!pip install torchtext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.9 MB/s eta 0:00:0000:0100:01
  Using cached torch-2.0.0-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0.dev20230408
    Uninstalling torch-2.1.0.dev20230408:
      Successfully uninstalled torch-2.1.0.dev20230408


In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold
import pandas as pd
from itertools import chain
from tqdm import tqdm




In [110]:
df_train = pd.read_csv('dataset/train.csv', nrows=80000).fillna('')
df_test = pd.read_csv('dataset/test.csv').fillna('')
# df_sample = pd.read_csv('dataset/sample_submission.csv')

In [111]:
#ent_df = pd.read_csv("entities_from_all_text_40000_just_text.csv")

In [112]:
#ent_df
#ent_df=ent_df.drop("TITLE",axis = "columns")
#ent_df=ent_df.drop("Unnamed: 0",axis = "columns")


In [113]:
#ent_df

In [114]:
y = df_train['PRODUCT_LENGTH'].astype(int)

#X = ent_df
X = pd.DataFrame()
X['PRODUCT_TYPE_ID'] = df_train['PRODUCT_TYPE_ID']
X['TEXT'] =  df_train['TITLE'] + ' ' + df_train['DESCRIPTION'] + ' ' + df_train['BULLET_POINTS']


In [115]:
X = X.to_numpy()

In [116]:
X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.01, random_state=42)

In [117]:
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

In [118]:
transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

# Training config
kfold = KFold(n_splits=7)# StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.01, random_state=42)

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(X_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 7 folds for each of 20 candidates, totalling 140 fits


In [119]:
X_test = pd.DataFrame()
X_test['PRODUCT_TYPE_ID'] = df_test['PRODUCT_TYPE_ID']
X_test['TEXT'] =  df_test['TITLE'] + ' ' + df_test['DESCRIPTION'] + ' ' + df_test['BULLET_POINTS']

y_predicted = []

# batches of 10,000:
test_batch_size = 10000
for i in range(0, len(X_test) + test_batch_size, test_batch_size):
    try:
        y_predicted.append(rf_model.predict(X_test.iloc[i:i+test_batch_size].to_numpy()))
    except:
        pass

ans = pd.DataFrame()
ans['PRODUCT_ID'] = df_test['PRODUCT_ID']
ans['PRODUCT_LENGTH'] = list(chain(*y_predicted))

ans.to_csv('submission_sklearn.csv', index=False)
ans

,PRODUCT_ID,PRODUCT_LENGTH
0,604373,600
1,1729783,600
2,1871949,600
3,1107571,600
4,624253,600
...,...,...
734731,921419,600
734732,2456362,600
734733,841529,600
734734,1190194,600


In [11]:
# y_predicted_best = []

# # batches of 10,000:
# test_batch_size = 10000
# for i in range(0, len(X_test) + test_batch_size, test_batch_size):
#     y_predicted_best.append(rf_best.predict(X_test.iloc[i:i+test_batch_size].to_numpy()))

In [12]:
# ans2 = pd.DataFrame()
# ans2['PRODUCT_ID'] = df_test['PRODUCT_ID']
# ans2['PRODUCT_LENGTH'] = list(chain(*y_predicted_best))

# ans2.to_csv('submission_sklearn_best.csv', index=False)
# ans2

## With Bert


In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from collections import Counter
import pandas as pd
from itertools import chain
from tqdm import tqdm


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop = set(stopwords.words('english'))

X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.01, random_state=42)

X_train_num = [row[:-2].astype(float) for row in X_train]
X_train_text = [row[-1] for row in X_train]

tokenized = []
wnl = nltk.stem.WordNetLemmatizer()
for text in tqdm(X_train_text):
    # Filter alphabet words only and non stop words, make it loser case
    words = [word.lower() for word in word_tokenize(text) if ((word.isalpha()==1) & (word not in stop))]
    # Lemmatize words 
    tokens = [wnl.lemmatize(wnl.lemmatize(word, 'n'), 'v') for word in words]
    tokenized.append(tokens)

all_words = [word for text in tokenized for word in text]
counts = Counter(all_words)
bow = sorted(counts, key=counts.get, reverse=True)
vocab = {word: ii for ii, word in enumerate(counts, 1)}
id2vocab = {v: k for k, v in vocab.items()}
token_ids = [[vocab[word] for word in text_words] for text_words in tokenized]


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/software/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/software/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/software/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|███████████████████████████████████| 99000/99000 [00:56<00:00, 1758.94it/s]


In [47]:
# Build a dataloader
def dataloader(text_inputs, num_inputs, labels, sequence_length=200, batch_size=16, shuffle=False):
    if shuffle:
        indices = list(range(len(text_inputs)))
        random.shuffle(indices)
        text_inputs = [text_inputs[idx] for idx in indices]
        num_inputs = [num_inputs[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(text_inputs)

    for ii in range(0, total_sequences, batch_size):
        batch_texts = text_inputs[ii: ii+batch_size]        
        batch = torch.zeros((sequence_length, len(batch_texts)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_texts):
            token_tensor = torch.tensor(tokens)
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        label_tensor = torch.tensor(labels[ii: ii+len(batch_texts)])
        num_tensor = torch.tensor(num_inputs[ii: ii+len(batch_texts)])
        
        yield batch, num_tensor, label_tensor

In [62]:
from torch import nn

# Model taking non-text and text inputs as well
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, dense_size, num_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model
        """
        super().__init__()
        self.vocab_size = vocab_size
        
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(lstm_size, dense_size)
        self.fc2 = nn.Linear(dense_size + num_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """
        Initialize the hidden state
        """
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

    def forward(self, nn_input_text, nn_input_num, hidden_state):
        """
        Perform a forward pass of the model on nn_input
        """
        batch_size = nn_input_text.size(0)
        nn_input_text = nn_input_text.long()
        embeds = self.embedding(nn_input_text)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        # Stack up LSTM outputs, apply dropout
        lstm_out = lstm_out[-1,:,:]
        lstm_out = self.dropout(lstm_out)
        # Dense layer
        dense_out = self.fc1(lstm_out)
        # Concatinate the dense output and meta inputs
        concat_layer = torch.cat((dense_out, nn_input_num.float()), 1)
        out = self.fc2(concat_layer)
        logps = self.softmax(out)

        return logps, hidden_state

In [63]:
n_input_num = len(X_train_num[0])
y_train = y_train.tolist()

AttributeError: 'list' object has no attribute 'tolist'

In [64]:
import torch
from torch import optim

# Training parameters
epochs=3
batch_size=8
learning_rate=1e-4
sequence_length=200
clip=5

# Set model
model = TextClassifier(len(vocab)+1, 512, 128, 8, n_input_num, 3, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
# Set device
device = torch.device("cpu")
model.to(device)
# Set Loss Functoin and Optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Start Training
model.train()
for epoch in range(epochs):
    print("###### Epoch {}/{} #####".format(epoch+1, epochs))
    hidden = model.init_hidden(batch_size)
    # Training Step
    for text_batch, mum_batch, labels in tqdm(dataloader(token_ids, X_train_num, y_train, 
            batch_size=batch_size, sequence_length=sequence_length, shuffle=False),
                                              total=int(len(y_train)/batch_size)):
        # Skip the last batch of which size is not equal to batch_size
        if text_batch.size(1) != batch_size:
            break
        # Creating new variables for the hidden state to avoid backprop entire training history
        hidden = tuple([each.data for each in hidden])
        # Set Device
        #text_batch, mum_batch, labels = np.float32(text_batch),np.float32(mum_batch),np.float32(labels)
        text_batch, mum_batch, labels = text_batch.to(device), mum_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        model.zero_grad()
        output, hidden = model(text_batch, mum_batch, hidden)
        #loss = criterion(output, labels)
        #loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

    # Evaluation Step


###### Epoch 1/3 #####


100%|█████████████████████████████████████| 12375/12375 [05:21<00:00, 38.49it/s]


###### Epoch 2/3 #####


100%|█████████████████████████████████████| 12375/12375 [05:23<00:00, 38.25it/s]


###### Epoch 3/3 #####


100%|█████████████████████████████████████| 12375/12375 [05:24<00:00, 38.15it/s]


In [103]:
# Define the input text that you want to classify
input_text = "This is a positive review."

# Tokenize the input text using the Vocab object
input_tokens = [vocab.stoi(token, vocab.stoi['<unk>']) for token in input_text.split()]

# Pad the input tokens to a fixed length
padded_input_tokens = pad_sequences([input_tokens], maxlen=n_input_num, padding='post')

# Make a prediction using the model
prediction = model.predict(padded_input_tokens)

# Print the predicted class
print(prediction)


AttributeError: 'Vocab' object has no attribute 'stoi'

In [98]:
predict_numerical_values(model, "This a text of 10 times the power of the length of smae")

TypeError: convert_text_to_input_num() missing 1 required positional argument: 'sequence_length'

In [81]:
# Initialize mum_batch and hidden tensors
batch_size = 1
mum_batch = torch.zeros(batch_size, sequence_length).long()
hidden = tuple(h.to(device) for h in model.init_hidden(batch_size))


In [83]:
input_string = "This is an example input string."
preprocessed_data = preprocess_input(input_string, sequence_length, vocab)

# Initialize mum_batch and hidden tensors
batch_size = 1
mum_batch = torch.zeros(batch_size, sequence_length).long()
hhidden = tuple(h.to(device) for h in model.init_hidden(batch_size))# add to(device) to ensure that the tensor is on the correct device

# Pass preprocessed data through model
with torch.no_grad():
    output, hidden = model(preprocessed_data.unsqueeze(0), mum_batch, hidden)

# Convert output to labels
predicted_labels = torch.argmax(output, dim=1)

print("Predicted label:", predicted_labels.item())


AssertionError: LSTM: Expected input to be 2-D or 3-D but received 4-D tensor

In [51]:
model.eval()

TextClassifier(
  (embedding): Embedding(105362, 512)
  (lstm): LSTM(512, 128, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=128, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=3, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [ ]:
X_test = pd.DataFrame()
X_test['PRODUCT_TYPE_ID'] = df_test['PRODUCT_TYPE_ID']
X_test['TEXT'] =  df_test['TITLE'] + ' ' + df_test['DESCRIPTION'] + ' ' + df_test['BULLET_POINTS']


test =X_test['TEXT'].to_list()

In [ ]:
predictions=predict(model,test,1000)

In [26]:
X_test = pd.DataFrame()
X_test['PRODUCT_TYPE_ID'] = df_test['PRODUCT_TYPE_ID']
X_test['TEXT'] =  df_test['TITLE'] + ' ' + df_test['DESCRIPTION'] + ' ' + df_test['BULLET_POINTS']


y_predicted = []

# batches of 10,000:
test_batch_size = 10000
for i in range(0, len(X_test) , test_batch_size):
    try:
        value = torch.tensor(X_test.iloc[i:i+test_batch_size].to_numpy())
        y_predicted.append(model(value))
    except:
        pass

y_predicted

#ans1 = pd.DataFrame()
#ans1['PRODUCT_ID'] = df_test['PRODUCT_ID']
#ans1['PRODUCT_LENGTH'] = list(chain(*y_predicted))

#ans1.to_csv('submission_sklearn.csv', index=False)
#ans1

In [104]:


from transformers import BertTokenizer, BertModel

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Model definition
class BertTextClassifier(nn.Module):
    def __init__(self, hidden_size, dense_size, meta_size, output_size, dropout=0.1):
        """
        Initialize the model
        """
        super().__init__()
        self.output_size = output_size 
        self.dropout = dropout        
        self.bert = BertModel.from_pretrained('bert-base-uncased',  
                                        output_hidden_states=True,
                                        output_attentions=True)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.weights = nn.Parameter(torch.rand(13, 1))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, dense_size)
        self.fc2 = nn.Linear(dense_size + meta_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, nn_input_meta):
        """
        Perform a forward pass of the model on nn_input
        """
        all_hidden_states, all_attentions = self.bert(input_ids)[-2:]
        batch_size = input_ids.shape[0]
        ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, 768)
        atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
        atten = F.softmax(atten.view(-1), dim=0)
        feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])        
        # Dense layer
        dense_out = self.fc1(self.dropout(feature))
        # Concatinate the dense output and meta inputs
        concat_layer = torch.cat((dense_out, nn_input_meta.float()), 1)
        out = self.fc2(concat_layer)

        return out

<function torch.nn.modules.module.Module.eval(self: ~T) -> ~T>

In [109]:
def predict(model, text, meta):
    """
    Predict the class for the given input text and meta features
    """
    # Tokenize input text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    # Perform a forward pass on the model
    output = model(inputs['input_ids'], meta)
    # Get the predicted class index
    _, predicted = torch.max(output.data, 1)
    return predicted.item()



In [108]:
# Initialize the model and load the trained weights
model = BertTextClassifier(hidden_size=768, dense_size=256, meta_size=10, output_size=2, dropout=0.1)
model.load_state_dict(torch.load('model_weights.pth'))

# Define the input text and meta features
text = "This movie is terrible!"
meta = torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.float32)

# Predict the class for the input text
predicted_class = predict(model, text, meta)

# Print the predicted class
if predicted_class == 0:
    print("The model predicts that the input text is negative.")
else:
    print("The model predicts that the input text is positive.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: 'model_weights.pth'